In [37]:
!pip install vaderSentiment
import datasets
import requests
import numpy as np

In [107]:
dataset = datasets.load_dataset('imdb')

train_dataset = dataset['train']
test_dataset = dataset['test']

url = "https://raw.githubusercontent.com/cjhutto/vaderSentiment/master/vaderSentiment/vader_lexicon.txt"
response = requests.get(url)
text = response.text
lines = text.split("\n")
values = [line.split() for line in lines]
dict = {}
for obj in values:
    if len(obj) != 13:
        continue
    dict.update({obj[0]: obj[1]})

print(dict)

Found cached dataset imdb (C:/Users/$0NT000-3G88GUGFR0B8/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)
100%|██████████| 3/3 [00:00<00:00, 199.52it/s]


{'$:': '-1.5', '%)': '-0.4', '%-)': '-1.5', '&-:': '-0.4', '&:': '-0.7', '(%': '-0.9', "('-:": '2.2', "(':": '2.3', '((-:': '2.1', '(*': '1.1', '(-%': '-0.7', '(-*': '1.3', '(-:': '1.6', '(-:0': '2.8', '(-:<': '-0.4', '(-:o': '1.5', '(-:O': '1.5', '(-:{': '-0.1', '(-:|>*': '1.9', '(-;': '1.3', '(-;|': '2.1', '(8': '2.6', '(:': '2.2', '(:0': '2.4', '(:<': '-0.2', '(:o': '2.5', '(:O': '2.5', '(;': '1.1', '(;<': '0.3', '(=': '2.2', '(?:': '2.1', '(^:': '1.5', '(^;': '1.5', '(^;0': '2.0', '(^;o': '1.9', '(o:': '1.6', ")':": '-2.0', ")-':": '-2.1', ')-:': '-2.1', ')-:<': '-2.2', ')-:{': '-2.1', '):': '-1.8', '):<': '-1.9', '):{': '-2.3', ');<': '-2.6', '*)': '0.6', '*-)': '0.3', '*-:': '2.1', '*-;': '2.4', '*:': '1.9', '*<|:-)': '1.6', '*\\0/*': '2.3', '*^:': '1.6', ',-:': '1.2', "---'-;-{@": '2.3', '--<--<@': '2.2', '.-:': '-1.2', '..###-:': '-1.7', '..###:': '-1.9', '/-:': '-1.3', '/:': '-1.3', '/:<': '-1.4', '/=': '-0.9', '/^:': '-1.0', '/o:': '-1.4', '0-8': '0.1', '0-|': '-1.2', '0:)': 

In [110]:
def lowercase_punctuation(text):
    newtext = text.lower()
    # remove <br /> tags
    newtext = newtext.replace('<br />', '')
    # remove punctuation
    newtext = ''.join([c if c not in "\"#$%&()*+,./:;<=>?@[\]^_`{|}~" else ' ' for c in newtext])
    return newtext.split()

In [109]:
def count_pronouns(word_list):
    count = 0
    for word in word_list:
        if word in ["i", "you", "we"]:
            count += 1
    return count

In [108]:
def lexicon_word_counter(word_list):
    positive_count = 0
    negative_count = 0
    lower_lexicon_value = -1
    upper_lexicon_value = 1
    for word in word_list:
        if word in dict.keys():
            sentimental_value = dict[word]
            if float(sentimental_value) < lower_lexicon_value:
                negative_count += 1
            elif float(sentimental_value) > upper_lexicon_value:
                positive_count += 1
    return positive_count, negative_count

In [117]:
def vector_generator(dataset):
    resulting_vector = []
    for obj in dataset:
        word_list = lowercase_punctuation(obj['text'])
        check_no = 1 if word_list.__contains__('no') else 0
        nb_pronouns = count_pronouns(word_list)
        check_exclamation = 1 if word_list.__contains__('!') else 0
        log_nb_words = np.log(len(word_list))
        positive_count, negative_count = lexicon_word_counter(word_list)
        resulting_vector.append([check_no, nb_pronouns, check_exclamation, log_nb_words, positive_count, negative_count])
    return resulting_vector

In [118]:
print(vector_generator(train_dataset))

[[1, 7, 0, 5.655991810819852, 7, 5], [1, 2, 0, 5.365976015021851, 4, 4], [1, 0, 0, 4.48863636973214, 3, 3], [0, 3, 0, 4.770684624465665, 5, 5], [0, 10, 0, 5.733341276897746, 4, 11], [0, 2, 0, 4.795790545596741, 3, 6], [0, 4, 0, 4.736198448394496, 0, 4], [0, 7, 0, 5.703782474656201, 17, 6], [1, 2, 0, 6.171700597410915, 32, 14], [1, 2, 0, 5.4116460518550396, 9, 8], [1, 7, 0, 5.552959584921617, 13, 4], [0, 3, 0, 4.912654885736052, 3, 4], [1, 4, 0, 4.875197323201151, 2, 5], [1, 3, 0, 4.941642422609304, 5, 2], [1, 14, 0, 5.8805329864007, 25, 10], [0, 5, 0, 5.308267697401205, 7, 8], [0, 2, 0, 4.3694478524670215, 7, 2], [0, 9, 0, 6.790097235513905, 40, 21], [0, 5, 1, 4.290459441148391, 6, 1], [0, 1, 0, 5.0238805208462765, 5, 11], [1, 7, 0, 5.135798437050262, 9, 2], [0, 3, 0, 5.049856007249537, 9, 4], [1, 4, 0, 5.075173815233827, 6, 6], [0, 4, 0, 5.777652323222656, 11, 16], [0, 3, 0, 4.624972813284271, 3, 6], [0, 5, 0, 5.204006687076795, 4, 6], [1, 2, 0, 5.389071729816501, 8, 10], [0, 4, 0, 4.